In [1]:
import vaex
import geopandas as gpd
from shapely.validation import make_valid
from difflib import SequenceMatcher
import numpy as np
import pandas as pd

In [3]:
dfs = []

for i in range(1995,2023):

    print(f'Processando {i}', end=', ')
    
    df = vaex.open(f'data/IPTU-HDF5/IPTU_{i}/IPTU_{i}.hdf5')
    # df = vaex.open(f'data//IPTU-HDF5/IPTU_{i}/IPTU_{i}.hdf5')

    df.setor = df['NUMERO DO CONTRIBUINTE'].str.slice(0,3)
    df.quadra = df['NUMERO DO CONTRIBUINTE'].str.slice(3,6)
    df.fillna(value='00-0', column_names=['NUMERO DO CONDOMINIO'], inplace=True)
    df.fillna(value=1., column_names=['FRACAO IDEAL'], inplace=True)
    df.fillna(value=0., column_names=['AREA CONSTRUIDA', 'AREA OCUPADA', 'AREA DO TERRENO'], inplace=True)
    df.sqlc = df.func.where(df['NUMERO DO CONDOMINIO'] == '00-0',
                                    df['NUMERO DO CONTRIBUINTE'].str.slice(0, 10) + '00',
                                    df['NUMERO DO CONTRIBUINTE'].str.slice(0, 6) + '0000' + df['NUMERO DO CONDOMINIO'].str.slice(0, 2))
    df['sq'] = df.setor + df.quadra
    df['sqlc'] = df.sqlc
    df_sqlc_ac = df.groupby('sqlc', agg={'area_contruida_total': vaex.agg.sum('AREA CONSTRUIDA')})
    df = df.join(df_sqlc_ac, on='sqlc')
    df['fracao_ideal'] = df.func.where(df['FRACAO IDEAL'] == 0.,  
                                                df['AREA CONSTRUIDA'] / df['area_contruida_total'],
                                                df['FRACAO IDEAL'])
    df['ca'] = (df['AREA CONSTRUIDA'] / df['fracao_ideal']) / df['AREA DO TERRENO']
    df['to'] = df['AREA OCUPADA'] / df['AREA DO TERRENO']

    df['valor_terreno'] = df['fracao_ideal'] * df['VALOR DO M2 DO TERRENO'] * df['AREA DO TERRENO']
    df['valor_construcao'] = df['VALOR DO M2 DE CONSTRUCAO'] * df['AREA CONSTRUIDA']

    dfs.append(df)

Processando 1995, Processando 1996, Processando 1997, Processando 1998, Processando 1999, Processando 2000, Processando 2001, Processando 2002, Processando 2003, Processando 2004, Processando 2005, Processando 2006, Processando 2007, Processando 2008, Processando 2009, Processando 2010, Processando 2011, Processando 2012, Processando 2013, Processando 2014, Processando 2015, Processando 2016, Processando 2017, Processando 2018, Processando 2019, Processando 2020, Processando 2021, Processando 2022, 

In [4]:
df_iptu = vaex.concat(dfs)

In [5]:
df_iptu.shape

(82322059, 37)

In [7]:
campos = ['NUMERO DO CONTRIBUINTE', 'sq', 'sqlc']

In [11]:
df_iptu['NUMERO DO CONTRIBUINTE'] = df_iptu['NUMERO DO CONTRIBUINTE'].str.replace('-', '')

In [19]:
df_iptu[campos]

#,NUMERO DO CONTRIBUINTE,sq,sqlc
0,00100300014,001003,001003000100
1,00100300022,001003,001003000200
2,00100300030,001003,001003000300
3,00100300049,001003,001003000400
4,00100300057,001003,001003000500
...,...,...,...
"82,322,054",31011802381,310118,310118000001
"82,322,055",31011802391,310118,310118000001
"82,322,056",31011802403,310118,310118000001
"82,322,057",31011900014,310119,310119000100


In [20]:
df_iptu['sq'] = df_iptu['sq'].astype('int64')

In [22]:
df_iptu['sqlc'] = df_iptu['sqlc'].astype('int64')

In [23]:
df_iptu[campos].dtypes

NUMERO DO CONTRIBUINTE    large_string
sq                               int64
sqlc                             int64
dtype: object

In [24]:
df_iptu[campos]

#,NUMERO DO CONTRIBUINTE,sq,sqlc
0,00100300014,1003,1003000100
1,00100300022,1003,1003000200
2,00100300030,1003,1003000300
3,00100300049,1003,1003000400
4,00100300057,1003,1003000500
...,...,...,...
"82,322,054",31011802381,310118,310118000001
"82,322,055",31011802391,310118,310118000001
"82,322,056",31011802403,310118,310118000001
"82,322,057",31011900014,310119,310119000100


In [26]:
df_iptu_agg = df_iptu[campos].groupby('NUMERO DO CONTRIBUINTE', agg={
    'sq':vaex.agg.first('sq'),
    'sqlc':vaex.agg.first('sqlc')
})

In [31]:
df_iptu_agg['sq'] = df_iptu_agg['sq'].astype('string').str.zfill(6)
df_iptu_agg['sqlc'] = df_iptu_agg['sqlc'].astype('string').str.zfill(12)

In [34]:
df_iptu_agg.to_pandas_df().to_feather('lotes_agregados_por_ano/sql_para_sqlc.feather')